# This script annotates transcripts of video simulation role-playing exercises from job candidates using large language models

In [18]:
# Install libraries that might not be installed as necessary (adjust manually) through Anaconda prompt

In [1]:
# Import necessary libraries
# import openai          # OpenAI API for AI model access
# from openai import OpenAI  # The main client class for interacting with OpenAI's API
from openai import AzureOpenAI
import requests        # HTTP library for making web requests (API calls, downloading data, etc.)
import re              # Regular expressions
import os              # Operating system interface
import pandas as pd    # DataFrames for tabular data manipulation and analysis
import numpy as np     # Numerical computing
import time            # Time-related functions
import random          # Random number generation
import getpass         # pop-up window for API completion
from tqdm import tqdm  # for showing progress bars

# set seed for reproducibility
random.seed(1337)  # Sets seed for Python's random module

In [34]:

# Load data
test_data = pd.read_excel("input/data_gitp_short_copy_for_prompt_testing.xlsx") # full advisement data
# test_data = pd.read_excel("input/data_gitp - Copy_for_data_exploration.xlsx") # n = 30 advisement participants
# test_data = test_data.iloc[:2] # keep only first few rows for pilot data analysis purposes
test_data.head()

# import prompts
from prompt_test.AD2_expl_en_c_20260108 import promptAD # manually change prompt name as desired from the /prompts directory
print("The number of keys in the dictionary is: ",len(promptAD)) # check that it correctly contains 4 'keys'

# # Advisement 2
# advise = pd.read_csv("data_advise_text.csv")
# # Change management 2
# change_manage = pd.read_csv("data_change_manage_text.csv")
# # Team management 2
# team_manage = pd.read_csv("data_team_manage_text.csv")
# # Team work 2
# team_work = pd.read_csv("data_team_work_text.csv")

The number of keys in the dictionary is:  4


In [ ]:
# print
# promptAD

{'AD_PROBL': ('AD_PROBL',
  '\n        **Context van de rollenspel-oefening:**\nJe bent een professionele recruiter. Je taak is om een sollicitant te evalueren op basis van hun antwoorden \nin een rollenspel-oefening. In deze oefening reageert de kandidaat verbaal op een fictieve collega genaamd Lara, \nverspreid over vier scènes. Lara deelt professionele frustraties, fouten bij klantprojecten, emotionele uitputting, \nen persoonlijke financiële zorgen, en vraagt de kandidaat om ondersteuning en advies. Samengevat:\n\n- **Scène 1**: Lara uit frustratie over het over het hoofd worden gezien op het werk, het uitvoeren van repetitieve taken, \nen het gebrek aan groeimogelijkheden. Ze vraagt om advies.\n- **Scène 2**: Lara bekent dat ze twee klantprojecten verkeerd heeft aangepakt, wat heeft geleid tot kostenproblemen en klantontevredenheid. \nZe is bang om het aan de teamleider te vertellen en vraagt of de kandidaat namens haar kan spreken.\n- **Scène 3**: Lara onthult emotionele uitputti

# Azure Foundry - API key

In [3]:
# Import Azure OpenAI API key
azure_api_key = getpass.getpass("Enter API key for Azure OpenAI: ") # manually enter API key to the pop-up window

# Import Azure OpenAI endpoint
azure_endpoint = getpass.getpass("Enter Azure OpenAI endpoint URL: ").strip() # manually enter endpoint URL

api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
)

# Open AI - API key

In [7]:
# Import OpenAI API key
gpt_api_key = getpass.getpass("Enter API key for OpenAI: ") # manually enter API key to the pop-up window
# Initialize the OpenAI client with your API key
client = OpenAI(api_key=gpt_api_key)

## Azure Foundry - Function to get rating from OpenAI

In [12]:
# Set default parameters
# model = "gpt-4.1-mini" # manually change the model
deployment = "gpt-4.1-mini" # manually change the model # This is the custom name you gave in Azure AI Studio, e.g., "my-gpt4-deployment"
temperature = 1      # manually change the temperature

# A - Function for explanation (number + explanation in the same cell)
def get_explanation(client, text, prompt):
    """Sends the text to GPT (Azure) and returns both score and explanation."""
    try:
        full_prompt = prompt.format(text=text)

        response = client.chat.completions.create(
            model=deployment,  # Azure deployment name
            messages=[
                {"role": "system", "content": "You are a professional recruiter."},
                {"role": "user", "content": full_prompt}
            ],
            temperature=temperature,
            max_completion_tokens=4000
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        print("Error:", e)
        return ""

# B - Function for numerical rating (1-5)
def get_rating(client, text, prompt):
    """Sends the text to GPT (Azure) and returns a numeric score (1–5, decimals allowed)."""
    try:
        full_prompt = prompt.format(text=text)

        response = client.chat.completions.create(
            model=deployment,
            messages=[
                {"role": "system", "content": "You are a professional recruiter."},
                {"role": "user", "content": full_prompt}
            ],
            temperature=temperature,
            max_completion_tokens=500
        )

        reply = response.choices[0].message.content.strip()

        import re
        match = re.search(r"\d+(\.\d+)?", reply)
        if match:
            return float(match.group())
        else:
            print("Warning: No numeric value found in reply:", reply)
            return None

    except Exception as e:
        print("Error:", e)
        return None

## Function to get ratings from openai API

In [29]:
# set default parameters
model = "gpt-5-mini" # manually change the model as desired
temperature = 1      # manually change the temperature as desired

# A - Function for explanation (number + explanation in the same cell)
def get_explanation(client, text, prompt):
    """Sends the text to GPT and returns both score and explanation."""
    try:
        response = client.responses.create(
            model=model,
            input=prompt.format(text=text),
            temperature=temperature
        )
        return response.output_text.strip()
    except Exception as e:
        print("Error:", e)
        return ""

# B - Function for numerical rating (1-5)
def get_rating(client, text, prompt):
    """Sends the text to GPT and returns a numeric score (1–5, decimals allowed)."""
    try:
        response = client.responses.create(
            model=model,
            input=prompt.format(text=text),
            temperature=temperature
        )
        reply = response.output_text.strip()

        # Extract any numeric value (with optional decimal part)
        import re
        match = re.search(r"\d+(\.\d+)?", reply)
        if match:
            value = float(match.group())  # keep decimals
            return value
        else:
            print("Warning: No numeric value found in reply:", reply)
            return None

    except Exception as e:
        print("Error:", e)
        return None

In [35]:
# Create empty dataframe to store the annotation results + explanation ONLY (but not numerical score)
# sed default parameters
model_short = "gpt4.1"
df = test_data
for code in promptAD.keys():
    df[f"{model_short}_{code}_explanation"] = None
    # df[f"{model_short}_{code}_score"] = None

print(test_data.columns) # make sure that correct empty columns have been created

Index(['VideoSimulatieType', 'AD_PROBL', 'AD_CREAT', 'AD_OORDE', 'AD_ORGANS',
       'AD_total', 'Beoordeling.all_nl_tw', 'Beoordeling.all_en_tw',
       'Beoordeling.all_en_tw_text_length', 'gpt4.1_AD_PROBL_explanation',
       'gpt4.1_AD_CREAT_explanation', 'gpt4.1_AD_OORDE_explanation',
       'gpt4.1_AD_ORGANS_explanation'],
      dtype='object')


In [ ]:
# sed default parameters
model_short = "gpt4.1" # manually change model as necessary
prompt_dict = promptAD # change prompt as necessary 
df = test_data # change data as necessary
# Prepare new columns for storing results
for code in prompt_dict.keys():
    df[f"{model_short}_{code}_explanation"] = None
    df[f"{model_short}_{code}_score"] = None

# print(test_data) # make sure that correct empty columns have been created 

# Text annotation

In [36]:
# Annotate ONLY verbal explanation + numerical value
# set default parameters
model_short = "gpt4.1" # manually change model as necessary
prompt_dict = promptAD # change prompt as necessary 
df = test_data # change data as necessary
# Loop through all rows
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['Beoordeling.all_en_tw']
    for code, (trait, prompt_expl) in prompt_dict.items():
        expl = get_explanation(client, text, prompt_expl)
        df.at[i, f"{model_short}_{code}_explanation"] = expl

100%|██████████| 117/117 [34:26<00:00, 17.67s/it]  


In [37]:
# Preview annotated df
print(df.head())

  VideoSimulatieType  AD_PROBL  AD_CREAT  AD_OORDE  AD_ORGANS  AD_total  \
0       Advisement_2       1.5      1.50  1.501667   2.333333   1.70875   
1       Advisement_2       1.0      2.00  2.000000   2.000000   1.75000   
2       Advisement_2       2.0      2.00  2.000000   2.000000   2.00000   
3       Advisement_2       2.0      2.00  2.000000   2.330000   2.08250   
4       Advisement_2       2.0      2.67  2.670000   1.000000   2.08500   

                               Beoordeling.all_nl_tw  \
0  Eerst, alles blijft tussen vier ogen dat wat v...   
1  Ja, nee, ik heb zeker wel verstand van deze za...   
2  Waar je op dit moment tegenaan loopt, en dan k...   
3  Is en waarmee kan ik je helpen en wat kunnen w...   
4  Oh, sorry nou leuk om je te ontmoeten. Ik ben ...   

                               Beoordeling.all_en_tw  \
0  First, everything stays between the NUMERIC_1 ...   
1  Yeah, no, I definitely do know about these thi...   
2  What you're running into right now, and t

In [38]:
# Extract the numerical value from the annotation and store in new columns
# List of existing explanation columns
explanation_cols = ['gpt4.1_AD_PROBL_explanation', 'gpt4.1_AD_CREAT_explanation',
                    'gpt4.1_AD_OORDE_explanation', 'gpt4.1_AD_ORGANS_explanation']

# Names of the new numeric columns to add
numeric_cols = [
    'gpt4.1_AD_PROBL', 'gpt4.1_AD_CREAT', 'gpt4.1_AD_OORDE','gpt4.1_AD_ORGANS']

# Extract numeric rating from the explanation columns into new columns
for expl_col, num_col in zip(explanation_cols, numeric_cols):
    df[num_col] = df[expl_col].str.extract(r'Rating:\s*([0-9]+(?:\.[0-9])?)')[0].astype(float)

# Compute the row-wise average of the four numeric columns and store in a new column
df['gpt4.1_AD_total'] = df[numeric_cols].mean(axis=1)

# df.head()

In [ ]:
# # # # # # # # # Optional: Save results MANUALLY change name of the file
# test_data.to_excel("output/AD2_en_annotated_c_20260108.xlsx", index = False)

In [40]:
# Get correlation matrix
# List of columns to include in the correlation matrix
corr_columns = ['AD_total', 'AD_PROBL', 'AD_CREAT', 'AD_OORDE', 'AD_ORGANS',
                'gpt4.1_AD_total', 'gpt4.1_AD_PROBL', 'gpt4.1_AD_CREAT', 'gpt4.1_AD_OORDE', 'gpt4.1_AD_ORGANS']

# Compute correlation matrix
corr_matrix = df[corr_columns].corr()

print(corr_matrix) # Display correlation matrix

                  AD_total  AD_PROBL  AD_CREAT  AD_OORDE  AD_ORGANS  \
AD_total          1.000000  0.884939  0.783539  0.818839   0.729383   
AD_PROBL          0.884939  1.000000  0.579549  0.738477   0.528567   
AD_CREAT          0.783539  0.579549  1.000000  0.532688   0.390970   
AD_OORDE          0.818839  0.738477  0.532688  1.000000   0.406225   
AD_ORGANS         0.729383  0.528567  0.390970  0.406225   1.000000   
gpt4.1_AD_total   0.561984  0.523606  0.450181  0.522149   0.319229   
gpt4.1_AD_PROBL   0.488360  0.482764  0.345035  0.444225   0.304297   
gpt4.1_AD_CREAT   0.516912  0.452293  0.483237  0.413767   0.311311   
gpt4.1_AD_OORDE   0.388751  0.374948  0.276067  0.435448   0.178953   
gpt4.1_AD_ORGANS  0.568544  0.508025  0.490821  0.511483   0.323743   

                  gpt4.1_AD_total  gpt4.1_AD_PROBL  gpt4.1_AD_CREAT  \
AD_total                 0.561984         0.488360         0.516912   
AD_PROBL                 0.523606         0.482764         0.452293   
AD_CR

In [ ]:
# Annotate BOTH (a) verbal explanation + numerica value, and (b) numerical value

# set default parameters
model_short = "gpt4.1" # manually change model as necessary
prompt_dict = promptAD # change prompt as necessary 
df = test_data # change data as necessary
# Loop through all rows
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['Beoordeling.all_en_tw']
    for code, (trait, prompt_expl, prompt_score) in prompt_dict.items():
        expl = get_explanation(client, text, prompt_expl)
        score = get_rating(client, text, prompt_score)
        df.at[i, f"{model_short}_{code}_explanation"] = expl
        df.at[i, f"{model_short}_{code}_score"] = score

# Preview annotated df
# print(df.head())

100%|██████████| 6/6 [07:33<00:00, 75.66s/it]


In [ ]:
# Optional: Save results
test_data.to_excel("en_annotated_test_data_20251112.xlsx", index = False)

# advise.to_csv('annotated_advise.csv', index=False)